In [1]:
import numpy
from collections import deque
import pandas
import math
import pandas_datareader.data as web
import datetime
import requests
import requests_cache
import xlrd
import tempfile
import itertools
import seaborn
from matplotlib import pyplot as plt

In [2]:
seaborn.set(style='whitegrid')
seaborn.set_context('poster')

In [3]:
def shiller_date_converter(x):
    year = int(x)
    month = int(x * 100) - (year * 100)
    dx = datetime.date(year, month, 1)
    return dx

def get_shiller(url="http://www.econ.yale.edu/~shiller/data/ie_data.xls"):
    expire_after = datetime.timedelta(days=3)
    session = requests_cache.CachedSession(cache_name='data-cache', backend='sqlite', expire_after=expire_after)

    r = session.get(url, stream=True)

    with tempfile.NamedTemporaryFile(suffix='.xls') as tmp:
        for chunk in r.iter_content(chunk_size=1024):
            tmp.write(chunk)
            
        tmp.flush()

        df = pandas.read_excel(tmp.name,
                               sheet_name='Data',
                               engine='xlrd',
                               skiprows=7,
                               skipfooter=11, # WARN: is this always 11?
                               index_col=0,
                               usecols="A:E,G:K")
        df.rename(columns={'Price' : 'Real Price',
                          'Dividend' : 'Real Dividend',
                          'Earnings' : 'Real Earnings'}, inplace=True)
        # Convert from Shiller's wonky date format to a real one
        df.index = pandas.DatetimeIndex([shiller_date_converter(n) for n in df.index])
    return df

shiller = get_shiller()
shiller.head()

PermissionError: [Errno 13] Permission denied: 'C:\\Users\\Justus\\AppData\\Local\\Temp\\tmpuv254rpv.xls'

In [ ]:
def get_tr(returns):
    value = 1
    shares = None
    s = {}
    for date, row in returns.iterrows():
        if not shares:
            # first loop
            shares = value / row['P']
        else:
            dividend = row['D'] / 12 * shares
            shares += dividend / row['P']
            value = shares * row['P']
        s[date] = value
    return pandas.Series(s)

In [ ]:
def get_price(returns):
    value = 1
    shares = None
    s = {}
    for date, row in returns.iterrows():
        if not shares:
            # first loop
            shares = value / row['P']
        else:
            dividend = row['D'] / 12 * shares
            value = shares * row['P']
        s[date] = value
    return pandas.Series(s)

In [ ]:
def get_dividends(returns):
    value = 1
    shares = None
    s = {}
    for date, row in returns.iterrows():
        if not shares:
            # first loop
            shares = value / row['P']
        else:
            dividend = row['D'] / 12 * shares
            value += dividend
        s[date] = value
    return pandas.Series(s)

In [ ]:
year = '1926'
r = shiller[year:]
total_return = get_tr(r)
price_only = get_price(r)
div_only = get_dividends(r)
df = pandas.DataFrame(data={'total returns': total_return, 'price only': price_only})#, 'dividends only': div_only})
#df = pandas.DataFrame(data={'total returns': total_return, 'price only': price_only, 'dividends only': div_only})
#df = pandas.DataFrame(data={'total returns': total_return, 'dividends only': div_only})

In [ ]:
plt.figure(figsize=(11,9))
plt.title(f'Growth of $1 since {year}')
seaborn.lineplot(data=df)

In [ ]:
def relative_contribution(returns):
    total_return = get_tr(returns)
    price_only = get_price(returns)
    return price_only[-1] / total_return[-1]

def relative_contribution_divs(returns):
    total_return = get_tr(returns)
    div_only = get_dividends(returns)
    return div_only[-1] / total_return[-1]

In [ ]:
s = {}
length = 30
for i in range(shiller.index[0].year, shiller.index[-1].year - length):
    r = shiller[str(i):str(i+length)]
    s[i] = relative_contribution(r)
rc = pandas.Series(s)

In [ ]:
plt.figure(figsize=(11,9))
plt.title(f'Capital gains rel contr, {length}yr rolling')
seaborn.lineplot(data=rc)

In [ ]:
s = {}
length = 0
for i in range(shiller.index[0].year, shiller.index[-1].year - length):
    r = shiller[str(i):]
    s[i] = relative_contribution(r)
rc1 = pandas.Series(s)
plt.figure(figsize=(11,9))
plt.title('Relative contribution of price gains based on starting year')
seaborn.lineplot(data=rc1)

In [ ]:
s = {}
length = 0
for i in range(shiller.index[0].year, shiller.index[-1].year - length):
    r = shiller[str(i):]
    s[i] = relative_contribution_divs(r)
rc2 = pandas.Series(s)
plt.figure(figsize=(11,9))
plt.title('Relative contribution of dividends based on starting year')
seaborn.lineplot(data=rc2)

In [ ]:
plt.figure(figsize=(11,9))
plt.title('Relative contribution')
df = pandas.DataFrame(data={'Dividends': rc2, 'Capital Gains': rc1})
seaborn.lineplot(data=df)